# GitHub Issues Query

This notebook lists issues (tickets) from a GitHub repository.

Setup:
1. Duplicate `.env.example` to `.env` in this folder
2. Fill `GITHUB_TOKEN`, `GITHUB_OWNER`, `GITHUB_REPO` (and `GITHUB_API_URL` if Enterprise)
3. Run the cells



## Fetch all issues from the repository

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables from .env if present
load_dotenv(dotenv_path=Path(".env"), override=False)

GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
GITHUB_OWNER = os.getenv("GITHUB_OWNER")
GITHUB_REPO = os.getenv("GITHUB_REPO")
GITHUB_API_URL = os.getenv("GITHUB_API_URL", "https://api.github.com")

missing = [name for name, val in {
    "GITHUB_TOKEN": GITHUB_TOKEN,
    "GITHUB_OWNER": GITHUB_OWNER,
    "GITHUB_REPO": GITHUB_REPO,
}.items() if not val]

if missing:
    raise RuntimeError(f"Missing required env vars: {', '.join(missing)}. Copy .env.example to .env and fill them.")

session = requests.Session()
session.headers.update({
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    "X-GitHub-Api-Version": "2022-11-28",
})

repo_issues_url = f"{GITHUB_API_URL}/repos/{GITHUB_OWNER}/{GITHUB_REPO}/issues"


def fetch_issues(state: str = "open", labels: str | None = None, per_page: int = 50, max_pages: int = 5):
    params = {"state": state, "per_page": per_page}
    if labels:
        params["labels"] = labels

    issues = []
    url = repo_issues_url
    for _ in range(max_pages):
        resp = session.get(url, params=params)
        resp.raise_for_status()
        page_items = resp.json()
        # GitHub returns PRs in this endpoint as well; filter to issues only
        page_issues = [it for it in page_items if "pull_request" not in it]
        issues.extend(page_issues)
        # Pagination via Link header
        next_url = None
        if "link" in resp.headers:
            for part in resp.headers["link"].split(","):
                seg, rel = part.split(";")
                if 'rel="next"' in rel:
                    next_url = seg.strip()[1:-1]
                    break
        if not next_url:
            break
        url, params = next_url, None  # next already encodes params
    return issues

issues = fetch_issues(state="open", labels=None, per_page=50, max_pages=5)
print(f"Fetched {len(issues)} issues from {GITHUB_OWNER}/{GITHUB_REPO}")

df = pd.DataFrame([
    {
        "number": it.get("number"),
        "title": it.get("title"),
        "state": it.get("state"),
        "created_at": it.get("created_at"),
        "updated_at": it.get("updated_at"),
        "user": (it.get("user") or {}).get("login"),
        "assignees": ",".join([a.get("login") for a in (it.get("assignees") or [])]),
        "labels": ",".join([l.get("name") for l in (it.get("labels") or [])]),
        "url": it.get("html_url"),
    }
    for it in issues
])

df.head(10)


Fetched 43 issues from TheSoftwareDevGuild/TheGuildGenesis


,number,title,state,created_at,updated_at,user,assignees,labels,url
0,142,Add contributor leaderboard,open,2025-11-24T15:04:07Z,2025-11-24T15:04:42Z,joelamouche,,"good first issue,front end,react,typescript,80pts",https://github.com/TheSoftwareDevGuild/TheGuil...
1,141,"Add a new leaderboard page, with link in the s...",open,2025-11-24T15:01:00Z,2025-11-24T15:01:00Z,joelamouche,,"good first issue,front end,react,typescript,20...",https://github.com/TheSoftwareDevGuild/TheGuil...
2,140,Create top badge owner leaderboard,open,2025-11-24T14:59:35Z,2025-11-24T15:02:16Z,joelamouche,,"good first issue,front end,react,typescript,40pts",https://github.com/TheSoftwareDevGuild/TheGuil...
3,138,Public contributor leaderboard,open,2025-11-17T13:58:58Z,2025-11-24T15:04:19Z,joelamouche,joelamouche,"enhancement,front end,planning",https://github.com/TheSoftwareDevGuild/TheGuil...
4,137,Figure out sharing new badges on X,open,2025-11-17T13:58:23Z,2025-11-17T13:58:28Z,joelamouche,joelamouche,planning,https://github.com/TheSoftwareDevGuild/TheGuil...
5,135,Add projects to the Backend,open,2025-11-17T12:38:30Z,2025-11-27T12:15:32Z,joelamouche,pheobeayo,"good first issue,rust,back-end,160pts,db,hackt...",https://github.com/TheSoftwareDevGuild/TheGuil...
6,134,Project Page,open,2025-11-17T12:38:11Z,2025-11-27T10:58:18Z,joelamouche,joelamouche,"enhancement,planning",https://github.com/TheSoftwareDevGuild/TheGuil...
7,131,Add backend for distributions,open,2025-11-03T13:30:48Z,2025-11-24T10:10:14Z,joelamouche,Chesblaw,"good first issue,rust,back-end,typescript,db",https://github.com/TheSoftwareDevGuild/TheGuil...
8,125,Improve UX and design of theguild.dev,open,2025-10-28T11:23:19Z,2025-10-28T11:23:19Z,joelamouche,,"good first issue,ux,design,hacktoberfest",https://github.com/TheSoftwareDevGuild/TheGuil...
9,118,Badge enhancements,open,2025-10-17T08:35:11Z,2025-10-30T13:44:59Z,joelamouche,joelamouche,"enhancement,planning,hacktoberfest",https://github.com/TheSoftwareDevGuild/TheGuil...


In [2]:
# Fetch all closed tickets
closed_tickets = fetch_issues(state="closed")

## Analyze closed tickets

In [3]:
# analyze closed tickets
# we want to get the following stats about all closed tickets: 
# - how many different contributors, 
# - how many different closed tickets, 
# - how many tags associated with the different tickets (total sum)
# - total sum of contributions points associated with tickets (see tags)
import re
import json


def get_contributions_from_tags(tags):
    # filter out the tags that don't look like xpts (where x is a number)
    tags = [tag for tag in tags if re.match(r'^\d+pts$', tag.get("name"))]
    # We want to sum the numbers
    return sum(int(tag.get("name").split("pts")[0]) for tag in tags)

def analyze_closed_tickets(closed_tickets):
    
    # Initialize dictionaries to store unique contributors and tags
    contributors = set()
    tags = set()
    total_contributions = 0
    
    # Process each closed ticket
    for ticket in closed_tickets:
        # Get the user who closed the ticket
        user = ticket.get("user", {}).get("login")
        if user:
            contributors.add(user)
        
        # Get the tags associated with the ticket
        for label in ticket.get("labels", []):
            tags.add(label.get("name"))
            
        # Get the contributions points from the tags
        contributions = get_contributions_from_tags(ticket.get("labels", []))
        total_contributions += contributions
    
    return {
        "contributors": len(contributors),
        "tags": len(tags),
        "total_contribution_tokens": total_contributions,
        "number_of_closed_tickets": len(closed_tickets),
    }
# analyze
stats = analyze_closed_tickets(closed_tickets)

# pretty print stats
print(json.dumps(stats, indent=4))
    

{
    "contributors": 4,
    "tags": 28,
    "total_contribution_tokens": 1040,
    "number_of_closed_tickets": 42
}


## Generate Attestations from Issues


### fetch all issues

In [ ]:
import json
import re
from datetime import datetime
from collections import defaultdict

# Fetch all issues (both open and closed) to get complete picture
all_issues = fetch_issues(state="closed", per_page=100, max_pages=10)
print(f"Fetched {len(all_issues)} total issues (open + closed)")

Fetched 42 total issues (open + closed)

BADGES/LABELS THAT NEED TO BE CREATED (23):
  - back-end
  - blockchain
  - bug
  - db
  - design
  - discord-bot
  - documentation
  - foundry
  - front end
  - good first issue
  - hacktoberfest
  - in progress
  - jupyter-notebook
  - nodejs
  - onlydust-wave
  - planning
  - python
  - react
  - rust
  - solidity
  - typescript
  - ux
  - wagmi



### Extract Labels

In [ ]:
# Extract unique labels/badges from all issues
all_labels = set()
for issue in all_issues:
    for label in issue.get("labels", []):
        label_name = label.get("name", "")
        # Filter out point labels (e.g., "10pts", "5pts") as they're not badges
        if label_name and not re.match(r'^\d+pts$', label_name):
            all_labels.add(label_name)

badges_to_create = sorted(list(all_labels))
print(f"\n{'='*60}")
print(f"BADGES/LABELS THAT NEED TO BE CREATED ({len(badges_to_create)}):")
print(f"{'='*60}")
for badge in badges_to_create:
    print(f"  - {badge}")
print(f"{'='*60}\n")


### Get github handles from API

In [5]:
# Optionally fetch profiles from backend API to map GitHub usernames to addresses
BACKEND_API_URL = os.getenv("BACKEND_API_URL", "http://localhost:3000")
print(BACKEND_API_URL)
github_to_address = {}

try:
    profiles_resp = requests.get(f"{BACKEND_API_URL}/profiles", timeout=5)
    if profiles_resp.status_code == 200:
        profiles = profiles_resp.json()
        for profile in profiles:
            if profile.get("github_login"):
                github_to_address[profile["github_login"].lower()] = profile["address"]
        print(f"Loaded {len(github_to_address)} GitHub username -> address mappings from backend API")
    else:
        print(f"Backend API not available (status {profiles_resp.status_code}), will need manual address mapping")
except Exception as e:
    print(f"Could not fetch profiles from backend API ({e}), will need manual address mapping")

# Show unmapped assignees
all_assignees = set()
for issue in all_issues:
    for assignee in issue.get("assignees", []):
        username = assignee.get("login", "").lower()
        if username:
            all_assignees.add(username)

unmapped = [u for u in all_assignees if u not in github_to_address]
if unmapped:
    print(f"\nWarning: {len(unmapped)} assignees without address mapping:")
    for u in sorted(unmapped):
        print(f"  - {u}")
    print("These will be skipped in attestations.\n")


https://theguild-backend-e2df290d177e.herokuapp.com/
Loaded 1 GitHub username -> address mappings from backend API

  - oscarwroche
  - peteroche
  - pheobeayo
  - rainwaters11
  - teddy1792
  - tusharshah21
  - yash-1104github
These will be skipped in attestations.



### Generate all attestations

In [6]:
# Generate attestations from issues
# For each issue with assignees and labels, create attestations
attestations = []

for issue in all_issues:
    assignees = issue.get("assignees", [])
    labels = issue.get("labels", [])
    issue_number = issue.get("number")
    issue_title = issue.get("title", "")
    issue_url = issue.get("html_url", "")
    
    # Skip if no assignees or no labels
    if not assignees or not labels:
        continue
    
    # Process each assignee
    for assignee in assignees:
        github_username = assignee.get("login", "").lower()
        if not github_username:
            continue
            
        # Get Ethereum address for this GitHub user
        recipient_address = github_to_address.get(github_username)
        if not recipient_address:
            continue  # Skip if no address mapping
        
        # Create attestation for each label (badge) on this issue
        for label in labels:
            label_name = label.get("name", "")
            # Skip point labels
            if not label_name or re.match(r'^\d+pts$', label_name):
                continue
            
            # Create justification based on issue
            justification = f"Awarded for contributions to issue #{issue_number}: {issue_title}"
            if issue_url:
                justification += f" ({issue_url})"
            
            attestations.append({
                "recipient": recipient_address,
                "badgeName": label_name,
                "justification": justification
            })

print(f"Generated {len(attestations)} attestations from {len(all_issues)} issues")
print(f"Unique recipients: {len(set(a['recipient'] for a in attestations))}")
print(f"Unique badges: {len(set(a['badgeName'] for a in attestations))}")


Generated 27 attestations from 42 issues
Unique recipients: 1
Unique badges: 13


### Save attestations to file

In [7]:
# Save attestations to date-based file
from pathlib import Path

output_dir = Path("../the-guild-smart-contracts")
output_dir.mkdir(parents=True, exist_ok=True)

# Create filename with current date
date_str = datetime.now().strftime("%Y-%m-%d")
output_file = output_dir / f"attestations-{date_str}.json"

output_data = {
    "attestations": attestations
}

with open(output_file, "w") as f:
    json.dump(output_data, f, indent=2)

print(f"\n✓ Saved {len(attestations)} attestations to: {output_file}")
print(f"  File size: {output_file.stat().st_size} bytes")



✓ Saved 27 attestations to: ../the-guild-smart-contracts/attestations-2025-12-02.json
  File size: 7751 bytes
